In [138]:
import requests
import pandas as pd
import numpy as np

In [139]:
mesReferencia = '1' #['1','2','3','4','5','6','7','8','9','10','11','12']
anoReferencia = '2023' #['2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']


In [140]:
requisicao = requests.get('https://sistemas.maua.sp.gov.br/Portaldatransparencia/API/Servidores/' + mesReferencia + '/' + anoReferencia)

servidores = requisicao.json()
df = pd.DataFrame(servidores)

In [141]:
#for ano in anoReferencia:
#    for mes in mesReferencia:
#        requisicao = requests.get('https://sistemas.maua.sp.gov.br/Portaldatransparencia/API/Servidores/' + mes + '/' + ano)

#        servidores = requisicao.json()
#        df = pd.DataFrame(servidores)
        
#        df.to_csv('A:/code/analiseOrcamentoMaua/servidoresPrefMaua/dados' + mes + '_' + ano + '.csv')
        

In [142]:
df.describe()

,cadastro,nome,secretaria,proventos,liquido,situacao,vinculo,cargo,localabonocovid,abonocovid,horaextracovid
count,7179,7179,7179,7179,7179,7179,7179,7179,7179,7179,7179
unique,7179,7082,48,4554,6420,4,18,281,2,20,57
top,41146,ANA PAULA DE SOUZA,Secretaria de Educação,2012,1972.94,Ativo,ESTAT EFETIVO,PEB I,N,0,0
freq,1,3,3061,261,154,7046,4922,1292,5580,5608,7119


In [143]:
df.dtypes

cadastro           object
nome               object
secretaria         object
proventos          object
liquido            object
situacao           object
vinculo            object
cargo              object
localabonocovid    object
abonocovid         object
horaextracovid     object
dtype: object

In [144]:
df['liquido'] = df['liquido'].astype(float)
df['proventos'] = df['proventos'].astype(float)
df['cadastro'] = df['cadastro'].astype(int)
df['abonocovid'] = df['abonocovid'].astype(int)
df['horaextracovid'] = df['horaextracovid'].astype(float)

In [145]:
df.dtypes

cadastro             int32
nome                object
secretaria          object
proventos          float64
liquido            float64
situacao            object
vinculo             object
cargo               object
localabonocovid     object
abonocovid           int32
horaextracovid     float64
dtype: object

In [146]:
prefeito = df['cargo']=='PREFEITO'

In [147]:
df[prefeito]

,cadastro,nome,secretaria,proventos,liquido,situacao,vinculo,cargo,localabonocovid,abonocovid,horaextracovid
4188,38797,MARCELO OLIVEIRA,Gabinete do Prefeito,18576.09,13701.04,Ativo,CARGO ELETIVO (PREF),PREFEITO,N,0,0.0


In [162]:
tetoMunicipal = df[prefeito].iloc[0]['proventos']
liquidoPrefeito = df[prefeito].iloc[0]['liquido']

filtroProventos = df['proventos'] > tetoMunicipal
filtroLiquido = df['liquido'] > liquidoPrefeito

In [163]:
df[filtroProventos].head()

,cadastro,nome,secretaria,proventos,liquido,situacao,vinculo,cargo,localabonocovid,abonocovid,horaextracovid
139,20183,ADRIANO PACIENTE GONCALVES,Controladoria Geral do Município,20633.62,7870.72,Ativo,ESTAT COMISS,CONTROLADOR-GERAL,N,0,0.0
282,12926,ALEXANDRE MARCOS SICILIANO JUNIOR,Secretaria de Saúde,20358.13,12177.21,Ativo,ESTAT EFETIVO,MEDICO-PEDIATRA 24H,S,300,0.0
287,22727,ALEXANDRE SILVA,Secretaria de Finanças,25408.55,18493.81,Ativo,ESTAT EFETIVO,AUD.FISC.TRIB. MUNICIPAIS II,N,0,0.0
561,21246,ANDRE DE ARIMATEIA DE SOUZA LINO,Secretaria de Saúde,22872.57,14778.95,Ativo,ESTAT EFETIVO,MEDICO-CIRURGIAO GERAL 40H,S,300,0.0
582,18409,ANDREA CHAVES FERREIRA SANTO AZZOLINI,Secretaria de Saúde,19907.31,13753.18,Ativo,ESTAT EFETIVO,MEDICO-GINECOLOGISTA 40H,S,300,0.0


In [151]:
furadoresTeto = df[filtroProventos]

In [152]:
furadoresTeto.count()

cadastro           51
nome               51
secretaria         51
proventos          51
liquido            51
situacao           51
vinculo            51
cargo              51
localabonocovid    51
abonocovid         51
horaextracovid     51
dtype: int64

In [153]:
furadoresTeto.describe()

,cadastro,proventos,liquido,abonocovid,horaextracovid
count,51.000000,51.000000,51.000000,51.000000,51.0
mean,19167.960784,22860.940196,14851.125098,100.000000,0.0
std,9151.955533,4252.679621,3168.721924,142.828569,0.0
min,2118.000000,18638.500000,7870.720000,0.000000,0.0
25%,12743.000000,19763.120000,13443.490000,0.000000,0.0
50%,18394.000000,20999.710000,13805.310000,0.000000,0.0
75%,25503.500000,24966.760000,15603.455000,300.000000,0.0
max,41021.000000,32975.340000,22313.830000,300.000000,0.0


In [154]:
procuradoriaGeral = furadoresTeto[furadoresTeto['cargo'] == 'PROCURADOR']

In [155]:
furadoresTeto = furadoresTeto.drop(procuradoriaGeral.index)


In [156]:
furadoresTeto.count()

cadastro           47
nome               47
secretaria         47
proventos          47
liquido            47
situacao           47
vinculo            47
cargo              47
localabonocovid    47
abonocovid         47
horaextracovid     47
dtype: int64

In [94]:
furadoresTeto = furadoresTeto[['nome','proventos','liquido','secretaria','cadastro','cargo']]
furadoresTeto.columns = "nome servidor", "salario bruto", "salario liquido", "secretaria", "cadastro", "cargo"

In [158]:
furadoresTeto.head()

,cadastro,nome,secretaria,proventos,liquido,situacao,vinculo,cargo,localabonocovid,abonocovid,horaextracovid
139,20183,ADRIANO PACIENTE GONCALVES,Controladoria Geral do Município,20633.62,7870.72,Ativo,ESTAT COMISS,CONTROLADOR-GERAL,N,0,0.0
282,12926,ALEXANDRE MARCOS SICILIANO JUNIOR,Secretaria de Saúde,20358.13,12177.21,Ativo,ESTAT EFETIVO,MEDICO-PEDIATRA 24H,S,300,0.0
287,22727,ALEXANDRE SILVA,Secretaria de Finanças,25408.55,18493.81,Ativo,ESTAT EFETIVO,AUD.FISC.TRIB. MUNICIPAIS II,N,0,0.0
561,21246,ANDRE DE ARIMATEIA DE SOUZA LINO,Secretaria de Saúde,22872.57,14778.95,Ativo,ESTAT EFETIVO,MEDICO-CIRURGIAO GERAL 40H,S,300,0.0
582,18409,ANDREA CHAVES FERREIRA SANTO AZZOLINI,Secretaria de Saúde,19907.31,13753.18,Ativo,ESTAT EFETIVO,MEDICO-GINECOLOGISTA 40H,S,300,0.0


In [133]:
furadoresTeto.to_csv(r'A:\code\analiseOrcamentoMaua\servidoresPrefMaua\furadoresTeto.csv')

Existe servidores duplicados?

In [134]:
servidorCadNome = df[['cadastro','nome','situacao','vinculo','secretaria','cargo','proventos','liquido']]

In [135]:
servidorCadNome.head()

,cadastro,nome,situacao,vinculo,secretaria,cargo,proventos,liquido
0,41146,ABELARDO DE LIMA RODRIGUES,Ativo,ESTAT EFETIVO,Secretaria de Governo,ASSISTENTE ADM I,3816.69,3315.78
1,33545,ABILENE SILVA NOGUEIRA,Ativo,ESTAT EFETIVO,Secretaria de Educação,AUX. DESENV. INFANTIL IV,3499.80,2315.94
2,39269,ABRAAO FRANCISCO DA COSTA,Ativo,ESTAT COMISS,Gabinete do Prefeito,ASSESSOR ESPECIAL,10416.46,7757.13
3,35744,ACACIO JOARI BELLI,Ativo,ESTAT EFETIVO,Secretaria de Planejamento Urbano,A.F.A.U. MUNICIPAIS I,7070.49,5611.92
4,27587,ADALBERTO APARECIDO DA SILVA SOUSA,Ativo,ESTAT EFETIVO,Secretaria de Assistência Social,AGENTE ADMINISTRATIVO I,5791.88,4727.84


In [136]:
semRepedidos = servidorCadNome['nome'].drop_duplicates().count()
todos = servidorCadNome['nome'].count()

print(todos - semRepedidos)

82


In [137]:
servidoresDuplicados = servidorCadNome.duplicated('nome', False)


In [138]:
servidorCadNome = servidorCadNome.assign(repetido = servidoresDuplicados)

In [139]:
servidorCadNome.head()

,cadastro,nome,situacao,vinculo,secretaria,cargo,proventos,liquido,repetido
0,41146,ABELARDO DE LIMA RODRIGUES,Ativo,ESTAT EFETIVO,Secretaria de Governo,ASSISTENTE ADM I,3816.69,3315.78,False
1,33545,ABILENE SILVA NOGUEIRA,Ativo,ESTAT EFETIVO,Secretaria de Educação,AUX. DESENV. INFANTIL IV,3499.80,2315.94,False
2,39269,ABRAAO FRANCISCO DA COSTA,Ativo,ESTAT COMISS,Gabinete do Prefeito,ASSESSOR ESPECIAL,10416.46,7757.13,False
3,35744,ACACIO JOARI BELLI,Ativo,ESTAT EFETIVO,Secretaria de Planejamento Urbano,A.F.A.U. MUNICIPAIS I,7070.49,5611.92,False
4,27587,ADALBERTO APARECIDO DA SILVA SOUSA,Ativo,ESTAT EFETIVO,Secretaria de Assistência Social,AGENTE ADMINISTRATIVO I,5791.88,4727.84,False


In [140]:
repetido = servidorCadNome['repetido']== True
servidoresRepetidos = servidorCadNome[repetido]


In [141]:
servidoresRepetidos['nome'].count()

162

In [144]:
servidoresRepetidos.head()

,cadastro,nome,situacao,vinculo,secretaria,cargo,proventos,liquido,repetido
63,41308,ADRIANA BATISTA SOUZA DOS SANTOS,Outra,ESTAT COMISS,Secretaria de Saúde,CHEFE DE EQUIPAMENTO DE SAÚDE,9687.52,6826.29,True
64,18126,ADRIANA BATISTA SOUZA DOS SANTOS,Ativo,ESTAT EFETIVO,Secretaria de Saúde,ENFERMEIRO,848.74,848.74,True
107,8007874,ADRIANA RODRIGUES DA SILVA,Ativo,EVENTUAL,Secretaria de Educação,PEB I,3340.06,2497.66,True
108,33379,ADRIANA RODRIGUES DA SILVA,Ativo,ESTAT EFETIVO,Secretaria de Educação,AUX. DESENV. INFANTIL III,3357.47,2642.10,True
328,28445,ALINE MIRON SANTOS,Ativo,ESTAT EFETIVO,Secretaria de Educação,PEB I,176.35,165.47,True


In [147]:
servidoresRepetidos = servidoresRepetidos[['cadastro','nome','situacao','cargo','vinculo','secretaria','proventos','liquido']]
servidoresRepetidos.columns = [['cadastro','nome servidor','situacao','cargo','vinculo','secretaria','salario bruto','salario liquido']]

In [148]:
servidoresRepetidos.head()

,cadastro,nome servidor,situacao,cargo,vinculo,secretaria,salario bruto,salario liquido
63,41308,ADRIANA BATISTA SOUZA DOS SANTOS,Outra,CHEFE DE EQUIPAMENTO DE SAÚDE,ESTAT COMISS,Secretaria de Saúde,9687.52,6826.29
64,18126,ADRIANA BATISTA SOUZA DOS SANTOS,Ativo,ENFERMEIRO,ESTAT EFETIVO,Secretaria de Saúde,848.74,848.74
107,8007874,ADRIANA RODRIGUES DA SILVA,Ativo,PEB I,EVENTUAL,Secretaria de Educação,3340.06,2497.66
108,33379,ADRIANA RODRIGUES DA SILVA,Ativo,AUX. DESENV. INFANTIL III,ESTAT EFETIVO,Secretaria de Educação,3357.47,2642.10
328,28445,ALINE MIRON SANTOS,Ativo,PEB I,ESTAT EFETIVO,Secretaria de Educação,176.35,165.47


In [ ]:
servidoresRepetidos.to_csv(r'A:\code\analiseOrcamentoMaua\servidoresPrefMaua\servidoresRepetidos.csv')

In [34]:
servidoresRepetidos[servidoresRepetidos['situacao'] != 'Ativo']

,cadastro,nome,situacao,cargo,vinculo,secretaria,proventos,liquido
63,41308,ADRIANA BATISTA SOUZA DOS SANTOS,Outra,CHEFE DE EQUIPAMENTO DE SAÚDE,ESTAT COMISS,Secretaria de Saúde,9687.52,6826.29
411,41279,ANA CLAUDIA DA SILVA,Outra,CHEFE DE EQUIPAMENTO (EFETIVO),ESTAT COMISS,Secretaria de Assistência Social,4077.39,3265.00
1799,8007716,EDILIO NUNES DUARTE,Outra,PEB II,EVENTUAL,Secretaria de Educação,1170.24,1051.64
1865,38832,EDNA MARIA DA SILVA,Outra,ASSESSOR DE GABINETE,ESTAT COMISS,Gabinete da Secretaria de Saúde,8892.80,6319.09
2495,41309,FLAVIA BERTOLUCCI CESAR,Outra,COORDENADOR DO CRAS (EFETIVO),ESTAT COMISS,Secretaria de Assistência Social,13079.47,9094.98
2696,41349,GISELE BEZERRA DA SILVA COSTA MELO,Outra,COORDENADOR DO CRAS (EFETIVO),ESTAT COMISS,Secretaria de Assistência Social,7937.73,5475.91
3104,38815,JEFFERSON YOSHIO TEGOSHI,Outra,CORREGEDOR DA GCM,ESTAT COMISS,Secretaria de Segurança Pública,12386.82,8940.67
4056,3009955,MANOEL PEREIRA DA SILVA,Outra,BOLSISTA QUALIF. PROFISSIONAL,BOLSISTA QP,Secretaria de Serviços Urbanos,1932.00,1895.64
4192,39494,MARCIO AVELINO DOS SANTOS,Outra,CHEFE DE EQUIPAMENTO (EFETIVO),ESTAT COMISS,Secretaria de Esportes e Lazer,5075.51,4133.66
4471,30051,MARIA DE LOURDES MANTOVANI FAVERO,Aposentado,APOSENTADO,APOSENTADO,Externos,5410.34,2098.10


In [35]:
somarRepetidos = servidoresRepetidos[['cadastro','nome','secretaria','cargo','proventos','liquido']]
somarRepetidos.reset_index()

,index,cadastro,nome,secretaria,cargo,proventos,liquido
0,63,41308,ADRIANA BATISTA SOUZA DOS SANTOS,Secretaria de Saúde,CHEFE DE EQUIPAMENTO DE SAÚDE,9687.52,6826.29
1,64,18126,ADRIANA BATISTA SOUZA DOS SANTOS,Secretaria de Saúde,ENFERMEIRO,848.74,848.74
2,107,8007874,ADRIANA RODRIGUES DA SILVA,Secretaria de Educação,PEB I,3340.06,2497.66
3,108,33379,ADRIANA RODRIGUES DA SILVA,Secretaria de Educação,AUX. DESENV. INFANTIL III,3357.47,2642.10
4,328,28445,ALINE MIRON SANTOS,Secretaria de Educação,PEB I,176.35,165.47
...,...,...,...,...,...,...,...
157,6810,5001976,VANESSA APARECIDA CAVALCANTE,Secretaria de Educação,PEB I,2340.86,2051.86
158,6850,8007784,VANESSA ROSANI CRUZ,Secretaria de Educação,PEB I,715.07,526.54
159,6851,41253,VANESSA ROSANI CRUZ,Secretaria de Educação,PEB II - AEE,4925.58,4310.59
160,6994,32400,WASHINGTON LUIZ DA SILVA,Secretaria de Serviços Urbanos,AUX. APOIO OPERACIONAL I,3051.95,2376.97


In [36]:
i = 0
salariosLiquidos = []
salariosBrutos = []
servidoresNomes = []
secretaria = []
cadastro1 = []
cadastro2 = []
cargo1 = []
cargo2 = []

for linha in somarRepetidos['nome']:
    if(i< somarRepetidos['nome'].count()-1):
        if (somarRepetidos.iloc[i]['nome'] == somarRepetidos.iloc[i+1]['nome']):
            bruto = somarRepetidos.iloc[i]['proventos']+somarRepetidos.iloc[i+1]['proventos']
            liquido = somarRepetidos.iloc[i]['liquido']+somarRepetidos.iloc[i+1]['liquido']
            salariosBrutos.append(float(round(bruto,2)))
            salariosLiquidos.append(float(round(liquido,2)))
            servidoresNomes.append(str(somarRepetidos.iloc[i]['nome']))
            secretaria.append(str(somarRepetidos.iloc[i]['secretaria']))
            cadastro1.append(str(somarRepetidos.iloc[i]['cadastro']))
            cadastro2.append(str(somarRepetidos.iloc[i+1]['cadastro']))
            cargo1.append(str(somarRepetidos.iloc[i]['cargo']))
            cargo2.append(str(somarRepetidos.iloc[i+1]['cargo']))
    i = i + 1
    

In [37]:
salariosBrutos = np.array(salariosBrutos)
salariosLiquidos = np.array(salariosLiquidos)
servidoresNomes = np.array(servidoresNomes)
secretaria = np.array(secretaria)
cadastro1 = np.array(cadastro1)
cadastro2 = np.array(cadastro2)
cargo1 = np.array(cargo1)
cargo2 = np.array(cargo2)

In [38]:
servidoresNomes = pd.DataFrame(servidoresNomes, columns=['nome'])
salariosLiquidos = pd.DataFrame(salariosLiquidos, columns=['liquido'])
salariosBrutos = pd.DataFrame(salariosBrutos, columns=['bruto'])
secretaria = pd.DataFrame(secretaria, columns=['secretaria'])
cadastro1 = pd.DataFrame(cadastro1, columns=['cadastro1'])
cadastro2 = pd.DataFrame(cadastro2, columns=['cadastro2'])
cargo1 = pd.DataFrame(cargo1, columns=['cargo1'])
cargo2 = pd.DataFrame(cargo2, columns=['cargo2'])

In [39]:
servidoresRepedidosSomados = pd.merge(servidoresNomes,salariosBrutos, left_index= True , right_index= True)
servidoresRepedidosSomados = pd.merge(servidoresRepedidosSomados,salariosLiquidos, left_index= True , right_index= True)
servidoresRepedidosSomados = pd.merge(servidoresRepedidosSomados,secretaria, left_index= True , right_index= True)
servidoresRepedidosSomados = pd.merge(servidoresRepedidosSomados,cadastro1, left_index= True , right_index= True)
servidoresRepedidosSomados = pd.merge(servidoresRepedidosSomados,cadastro2, left_index= True , right_index= True)
servidoresRepedidosSomados = pd.merge(servidoresRepedidosSomados,cargo1, left_index= True , right_index= True)
servidoresRepedidosSomados = pd.merge(servidoresRepedidosSomados,cargo2, left_index= True , right_index= True)

In [40]:
servidoresRepedidosSomados.head()

,nome,bruto,liquido,secretaria,cadastro1,cadastro2,cargo1,cargo2
0,ADRIANA BATISTA SOUZA DOS SANTOS,10536.26,7675.03,Secretaria de Saúde,41308,18126,CHEFE DE EQUIPAMENTO DE SAÚDE,ENFERMEIRO
1,ADRIANA RODRIGUES DA SILVA,6697.53,5139.76,Secretaria de Educação,8007874,33379,PEB I,AUX. DESENV. INFANTIL III
2,ALINE MIRON SANTOS,2715.93,2444.14,Secretaria de Educação,28445,41318,PEB I,PEB II - AEE
3,ANA CLAUDIA DA SILVA,4845.43,3943.06,Secretaria de Assistência Social,35833,41279,AUX. APOIO OPERACIONAL I,CHEFE DE EQUIPAMENTO (EFETIVO)
4,ANA PAULA DE SOUZA,8501.74,6192.56,Secretaria de Saúde,19742,27044,AUXILIAR DE ENFERMAGEM,AGENTE COMUNITÁRIO DE SAÚDE


In [41]:
servidoresRepedidosSomados.columns = "Nome do Servidor","Salario Bruto","Salario Liquido", "Secretaria no 1º Cadastro", "1º Cadastro", "2º Cadastro","Cargo 1º Cad","Cargo 2º Cad"
servidoresRepedidosSomados.to_csv(r'A:\code\analiseOrcamentoMaua\servidoresPrefMaua\servidorRepetidos.csv')

In [42]:
filtroProventos = servidoresRepedidosSomados['Salario Bruto'] > proventoTeto
filtroLiquidos = servidoresRepedidosSomados['Salario Liquido'] > liquidoTeto

In [43]:
df_servidoresRepetidos_105_bruto = servidoresRepedidosSomados[filtroProventos]
df_servidoresRepetidos_105_liquido = servidoresRepedidosSomados[filtroLiquidos]

In [44]:
df_servidoresRepetidos_105_liquido.columns = "Nome do Servidor","Salario Bruto","Salario Liquido", "Secretaria no 1º Cadastro", "1º Cadastro", "2º Cadastro","Cargo 1º Cad","Cargo 2º Cad"
df_servidoresRepetidos_105_bruto.columns = "Nome do Servidor","Salario Bruto","Salario Liquido", "Secretaria no 1º Cadastro", "1º Cadastro", "2º Cadastro","Cargo 1º Cad","Cargo 2º Cad"

In [45]:
df_servidoresRepetidos_105_liquido.to_csv(r'A:\code\analiseOrcamentoMaua\servidoresPrefMaua\servidorLiquidos105.csv')


In [46]:
df_servidoresRepetidos_105_bruto

,Nome do Servidor,Salario Bruto,Salario Liquido,Secretaria no 1º Cadastro,1º Cadastro,2º Cadastro,Cargo 1º Cad,Cargo 2º Cad
7,ANDRE LUIZ FISCHER,23362.52,17206.61,Secretaria de Saúde,41037,40413,MÉDICO CLÍNICO GERAL,MÉDICO CLÍNICO GERAL
11,BIANCA DE PAIVA BATISTA DOS SANTOS,25562.52,18801.61,Secretaria de Saúde,40435,41202,MÉDICO CLÍNICO GERAL,MÉDICO CLÍNICO GERAL
14,CINCINATO LOURENCO FREIRE FILHO,23362.52,17206.61,Secretaria de Saúde,40411,41027,MÉDICO CLÍNICO GERAL,MÉDICO CLÍNICO GERAL
16,DANIEL GEORGE LIMA HAPP,32926.52,24140.51,Secretaria de Saúde,41029,41249,MÉDICO ESPECIALIDADES,MÉDICO ESPECIALIDADES
26,ELY PATRICIA TIQUISARA SARAVIA,34746.52,25460.01,Secretaria de Saúde,41237,40443,MÉDICO ESPECIALIDADES,MÉDICO ESPECIALIDADES
27,ESTEVAO BORGES JORGE,27362.52,20106.61,Secretaria de Saúde,41055,40403,MÉDICO ESPECIALIDADES,MÉDICO ESPECIALIDADES
30,FLAVIA FOTI,32562.52,23980.89,Secretaria de Saúde,41236,41021,MÉDICO ESPECIALIDADES,MÉDICO ESPECIALIDADES
31,FLAVIA KALLAS RODRIGUES DE CARVALHO,23362.52,17206.61,Secretaria de Saúde,41024,40400,MÉDICO CLÍNICO GERAL,MÉDICO CLÍNICO GERAL
39,LUCAS VINICIUS CORREIA DE OLIVEIRA,23362.52,16986.61,Secretaria de Saúde,41258,40414,MÉDICO CLÍNICO GERAL,MÉDICO CLÍNICO GERAL
41,MARCIA VALERIA LOZANO CONZ RODRIGUES,27473.89,16982.97,Secretaria de Saúde,18162,1000090,MEDICO-PEDIATRA 36H,MEDICO-20H


In [47]:
df_servidoresRepetidos_105_bruto.count()

Nome do Servidor             15
Salario Bruto                15
Salario Liquido              15
Secretaria no 1º Cadastro    15
1º Cadastro                  15
2º Cadastro                  15
Cargo 1º Cad                 15
Cargo 2º Cad                 15
dtype: int64